### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [40]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 4   # False for selecting word group according to max word group length

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

1


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [6]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [7]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [10]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
150,buradaydı,16726
151,buysa,16063
152,miydiniz,15468
153,bunlarla,15042


In [11]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,french_word
0,abajur,abat-jour
1,abaküs,abacus
2,abandone,abandonner
3,abiye,habillé
4,abluka,blocus
...,...,...
1593,yeniçeri,janissaire
1594,yoğurt,yaourt
1595,zebra,zèbre
1596,zombi,zombie


In [12]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,french_word
0,abajur,abat-jour
1,abaküs,abacus
2,abandoned,abandonner
3,abiye,habillé
4,abluka,blocus
...,...,...
5385,zombileri,zombie
5386,zombiye,zombie
5387,zürafa,girafe
5388,zürafalar,girafe


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [15]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

#### Youtube Sentence Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)

In [ ]:
df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [41]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [42]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [43]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [44]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [45]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [46]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [47]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [48]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [49]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [50]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,9.0,"sizlerede, yağ, eridi, ediyorum, sayılır, orta...","gibi, o, da, çok, onun, bugün, bunu, ama, de, ..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,4.4,"bütün, a, rahatsızız, önümüzdeki, dışında, izl...","gibi, genel, o, bir, vardır, teknik, burada, a..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"seviyorsunuzi, hem, isensiz, ioğluyla, h, isen...","benim, sen, biraz, de, ben, mi, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"kadının, baş, birbirinden, koyuyorsun, mehdi, ...","gibi, o, senin, bir, enerji, ama, kadar, mi, d..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.6,"moreover, a, accept, group, others, unable, he...","test, risk, not, prime, done"
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.3,"dostlarla, zamanla, a, dişleriniz, deposuyum, ...","gibi, sistemini, senin, bir, vitaminleri, parf..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"kızınız, uzaklaş, anlatabildim, dışında, nazlı...","gibi, o, birisi, yoksa, senden, de, ben, mi, v..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.7,"bütün, göre, ailece, kendi, salonumuz, kuşadas...","gene, buradaki, sitenin, kare, için, mobilyala..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"belgesellere, hikaye, küçük, hatta, fracchia, ...","bir, senaryoyu, bunun, film, ben, kadar, vardı..."


In [51]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,9.0,"sizlerede, yağ, eridi, ediyorum, sayılır, orta...","gibi, o, da, çok, onun, bugün, bunu, ama, de, ..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,4.4,"bütün, a, rahatsızız, önümüzdeki, dışında, izl...","gibi, genel, o, bir, vardır, teknik, burada, a..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"seviyorsunuzi, hem, isensiz, ioğluyla, h, isen...","benim, sen, biraz, de, ben, mi, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"kadının, baş, birbirinden, koyuyorsun, mehdi, ...","gibi, o, senin, bir, enerji, ama, kadar, mi, d..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.6,"moreover, a, accept, group, others, unable, he...","test, risk, not, prime, done"
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.3,"dostlarla, zamanla, a, dişleriniz, deposuyum, ...","gibi, sistemini, senin, bir, vitaminleri, parf..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"kızınız, uzaklaş, anlatabildim, dışında, nazlı...","gibi, o, birisi, yoksa, senden, de, ben, mi, v..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.7,"bütün, göre, ailece, kendi, salonumuz, kuşadas...","gene, buradaki, sitenin, kare, için, mobilyala..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"belgesellere, hikaye, küçük, hatta, fracchia, ...","bir, senaryoyu, bunun, film, ben, kadar, vardı..."


In [52]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,9.0,"sizlerede, yağ, eridi, ediyorum, sayılır, orta...","gibi, o, da, çok, onun, bugün, bunu, ama, de, ..."
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"seviyorsunuzi, hem, isensiz, ioğluyla, h, isen...","benim, sen, biraz, de, ben, mi, ne"
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"kadının, baş, birbirinden, koyuyorsun, mehdi, ...","gibi, o, senin, bir, enerji, ama, kadar, mi, d..."
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,10.1,"allahın, rahim, rahman, olan, suresi, i, adıyl...",ve
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,9.7,"mide, bütün, hazırlayabilirsiniz, suyumuzu, da...","gibi, o, bir, kart, ama, kadar, mi, bugün, içi..."
...,...,...,...,...,...
14043,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,20.1,"cevabını, bütün, soruların, yılbaşı, kanalıma,...","daha, ben, abone, videoyu, bu, için, ve, çok"
14044,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.3,"dostlarla, zamanla, a, dişleriniz, deposuyum, ...","gibi, sistemini, senin, bir, vitaminleri, parf..."
14045,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"kızınız, uzaklaş, anlatabildim, dışında, nazlı...","gibi, o, birisi, yoksa, senden, de, ben, mi, v..."
14046,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"belgesellere, hikaye, küçük, hatta, fracchia, ...","bir, senaryoyu, bunun, film, ben, kadar, vardı..."


In [53]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=True, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,video_id,sentence,word_ratio,different_word,intersect_word,search_string
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"kadının, baş, birbirinden, koyuyorsun, mehdi, ...","gibi, o, senin, bir, enerji, ama, kadar, mi, d...",onu ama ne kadar
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,9.7,"mide, bütün, hazırlayabilirsiniz, suyumuzu, da...","gibi, o, bir, kart, ama, kadar, mi, bugün, içi...",ve servis ama ama
8,01qjQ_KPgpY,mısınız ayrıntılar birazdan kaptan bahada herk...,5.2,"bütün, göre, a, kimsin, boyunca, dışında, yan,...","kültürü, buradaki, için, buranın, ben, de, tra...",bu pilotun ne kadar da
12,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,11.6,"edindiler, kendin, evlat, ablacığım, ediyorum,...","miyim, o, senin, bir, bisiklet, ama, kadar, mi...",miyim ben bu kadar
19,05ytKVIEV_I,herkese selamlar sevgiler kanalıma hoş geldin ...,7.1,"esra, a, sanırım, lezzeti, üzeri, kesmek, kal,...","gibi, o, bir, ama, yok, kadar, sende, rulet, b...",çikolatalı ve frambuaz gibi
...,...,...,...,...,...,...
14027,ztWGfBL1qLI,şöyle güçlü bir tarifle cildimi bembeyaz yapay...,5.5,"savaşarak, really, yağ, kendi, lekelerden, dil...","gibi, o, bir, burada, ama, kadar, keten, da, b...",vitamini var bu da
14030,ztt2Ge1EOd0,i nsanlar tarihleri boyunca renkleri nesneleri...,7.5,"kendi, boyunca, kadim, yutar, böyle, edilmiş, ...","içine, popüler, fantastik, sembolik, genel, o,...",sembolüdür bazen de bir
14037,zxDPt1mX8Vg,şimdi toyota gr suprayı bir adım öteye taşıyal...,16.2,"edeceğini, önümüzdeki, koltukları, merkezi, ye...","gibi, bir, kategorisi, ama, şasi, elektrik, fa...",bir elektronik kontrol ünitesi elektrik sistemi
14040,zyhG7Z4EAoA,herkese merhaba arkadaşlar kanalıma tekrar hoş...,5.3,"mide, çıkartarak, başa, işi, olduk, yapımında,...","gibi, o, bir, buradaki, ama, kadar, mi, fazla,...",mi evet bu da


In [54]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

,search_string,video_id,sentence
2,onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,ve servis ama ama,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...
8,bu pilotun ne kadar da,01qjQ_KPgpY,mısınız ayrıntılar birazdan kaptan bahada herk...
12,miyim ben bu kadar,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...
19,çikolatalı ve frambuaz gibi,05ytKVIEV_I,herkese selamlar sevgiler kanalıma hoş geldin ...
...,...,...,...
14027,vitamini var bu da,ztWGfBL1qLI,şöyle güçlü bir tarifle cildimi bembeyaz yapay...
14030,sembolüdür bazen de bir,ztt2Ge1EOd0,i nsanlar tarihleri boyunca renkleri nesneleri...
14037,bir elektronik kontrol ünitesi elektrik sistemi,zxDPt1mX8Vg,şimdi toyota gr suprayı bir adım öteye taşıyal...
14040,mi evet bu da,zyhG7Z4EAoA,herkese merhaba arkadaşlar kanalıma tekrar hoş...


In [55]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [56]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,start_time,end_time
0,onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
1,ve servis ama ama,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,0.000,59.370
2,bu pilotun ne kadar da,01qjQ_KPgpY,mısınız ayrıntılar birazdan kaptan bahada herk...,10.010,1127.030
3,miyim ben bu kadar,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,0.350,394.702
4,çikolatalı ve frambuaz gibi,05ytKVIEV_I,herkese selamlar sevgiler kanalıma hoş geldin ...,0.150,505.780
...,...,...,...,...,...
4921,vitamini var bu da,ztWGfBL1qLI,şöyle güçlü bir tarifle cildimi bembeyaz yapay...,0.768,482.047
4922,sembolüdür bazen de bir,ztt2Ge1EOd0,i nsanlar tarihleri boyunca renkleri nesneleri...,0.300,405.740
4923,bir elektronik kontrol ünitesi elektrik sistemi,zxDPt1mX8Vg,şimdi toyota gr suprayı bir adım öteye taşıyal...,0.280,143.840
4924,mi evet bu da,zyhG7Z4EAoA,herkese merhaba arkadaşlar kanalıma tekrar hoş...,0.000,610.910


In [57]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,onu ama ne kadar,5.004147,6.221372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
1,ve servis ama ama,58.528776,59.013117,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
2,bu pilotun ne kadar da,848.887751,851.067834,mısınız ayrıntılar birazdan kaptan bahada herk...,01qjQ_KPgpY
3,miyim ben bu kadar,315.741617,320.240784,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
4,çikolatalı ve frambuaz gibi,375.675165,378.674402,herkese selamlar sevgiler kanalıma hoş geldin ...,05ytKVIEV_I
...,...,...,...,...,...
4921,vitamini var bu da,156.816066,158.468242,şöyle güçlü bir tarifle cildimi bembeyaz yapay...,ztWGfBL1qLI
4922,sembolüdür bazen de bir,187.989181,190.167561,i nsanlar tarihleri boyunca renkleri nesneleri...,ztt2Ge1EOd0
4923,bir elektronik kontrol ünitesi elektrik sistemi,87.478138,91.675298,şimdi toyota gr suprayı bir adım öteye taşıyal...,zxDPt1mX8Vg
4924,mi evet bu da,87.035504,88.952586,herkese merhaba arkadaşlar kanalıma tekrar hoş...,zyhG7Z4EAoA


In [58]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,onu ama ne kadar,4.704147,6.521372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
1,ve servis ama ama,58.228776,59.313117,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
2,bu pilotun ne kadar da,848.587751,851.367834,mısınız ayrıntılar birazdan kaptan bahada herk...,01qjQ_KPgpY
3,miyim ben bu kadar,315.441617,320.540784,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
4,çikolatalı ve frambuaz gibi,375.375165,378.974402,herkese selamlar sevgiler kanalıma hoş geldin ...,05ytKVIEV_I
...,...,...,...,...,...
4921,vitamini var bu da,156.516066,158.768242,şöyle güçlü bir tarifle cildimi bembeyaz yapay...,ztWGfBL1qLI
4922,sembolüdür bazen de bir,187.689181,190.467561,i nsanlar tarihleri boyunca renkleri nesneleri...,ztt2Ge1EOd0
4923,bir elektronik kontrol ünitesi elektrik sistemi,87.178138,91.975298,şimdi toyota gr suprayı bir adım öteye taşıyal...,zxDPt1mX8Vg
4924,mi evet bu da,86.735504,89.252586,herkese merhaba arkadaşlar kanalıma tekrar hoş...,zyhG7Z4EAoA


In [59]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,onu ama ne kadar,5,7,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
1,ve servis ama ama,58,59,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
2,bu pilotun ne kadar da,849,851,mısınız ayrıntılar birazdan kaptan bahada herk...,01qjQ_KPgpY
3,miyim ben bu kadar,315,321,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
4,çikolatalı ve frambuaz gibi,375,379,herkese selamlar sevgiler kanalıma hoş geldin ...,05ytKVIEV_I
...,...,...,...,...,...
4921,vitamini var bu da,157,159,şöyle güçlü bir tarifle cildimi bembeyaz yapay...,ztWGfBL1qLI
4922,sembolüdür bazen de bir,188,190,i nsanlar tarihleri boyunca renkleri nesneleri...,ztt2Ge1EOd0
4923,bir elektronik kontrol ünitesi elektrik sistemi,87,92,şimdi toyota gr suprayı bir adım öteye taşıyal...,zxDPt1mX8Vg
4924,mi evet bu da,87,89,herkese merhaba arkadaşlar kanalıma tekrar hoş...,zyhG7Z4EAoA


In [60]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,bu mesajı çok çok çok çok çok çok çok çok çok ...,1297,1324,herkese merhaba sevgili transferler yine bir t...,S73oaKSsvtQ
1,daha daha daha daha dijital daha daha daha dah...,21,34,duyduklarınıza inanamayacaksınız arçelik servi...,iBHbG_CpBMg
2,bisküvi ve çikolata ekipmanları var bisküvi çi...,1038,1045,merhaba aslı şekerci ben öğretmenim aynı zaman...,LxuQYorE1jQ
3,bir şeyler bir şeyler prosedürü kumanda var bu...,740,749,bu şuradaki şeylere bakın bunların açık ne old...,komeAfuizRs
4,bir şey varmış o avukatlık ofisinde avukatlık ...,672,678,dünyanın en amatör ekipmanlarıyla kurulan mith...,B6APbgmIA6Q
...,...,...,...,...,...
4921,o o o o bu,915,919,o o e herkese selamlar ben savaş dönmez bugün ...,0OEXTH6p9zg
4922,da o da o,97,98,umut etmek inanmaktır öyle değil mi umut etmek...,Cd8YRWIc7d0
4923,o o o bu,1264,1274,evet efendim bugün solotürkün yani f alıyorlar...,RnY_tQs6OqA
4924,o o o da,613,616,o o e herkese selamlar ben savaş dönmez bugün ...,OhMefsPMa8A


In [61]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bu mesajı çok çok çok çok çok çok çok çok çok ...,1297,1324,herkese merhaba sevgili transferler yine bir t...,S73oaKSsvtQ,https://www.youtube.com/watch?v=S73oaKSsvtQ&t=...
1,daha daha daha daha dijital daha daha daha dah...,21,34,duyduklarınıza inanamayacaksınız arçelik servi...,iBHbG_CpBMg,https://www.youtube.com/watch?v=iBHbG_CpBMg&t=21s
2,bisküvi ve çikolata ekipmanları var bisküvi çi...,1038,1045,merhaba aslı şekerci ben öğretmenim aynı zaman...,LxuQYorE1jQ,https://www.youtube.com/watch?v=LxuQYorE1jQ&t=...
3,bir şeyler bir şeyler prosedürü kumanda var bu...,740,749,bu şuradaki şeylere bakın bunların açık ne old...,komeAfuizRs,https://www.youtube.com/watch?v=komeAfuizRs&t=...
4,bir şey varmış o avukatlık ofisinde avukatlık ...,672,678,dünyanın en amatör ekipmanlarıyla kurulan mith...,B6APbgmIA6Q,https://www.youtube.com/watch?v=B6APbgmIA6Q&t=...
...,...,...,...,...,...,...
4921,o o o o bu,915,919,o o e herkese selamlar ben savaş dönmez bugün ...,0OEXTH6p9zg,https://www.youtube.com/watch?v=0OEXTH6p9zg&t=...
4922,da o da o,97,98,umut etmek inanmaktır öyle değil mi umut etmek...,Cd8YRWIc7d0,https://www.youtube.com/watch?v=Cd8YRWIc7d0&t=97s
4923,o o o bu,1264,1274,evet efendim bugün solotürkün yani f alıyorlar...,RnY_tQs6OqA,https://www.youtube.com/watch?v=RnY_tQs6OqA&t=...
4924,o o o da,613,616,o o e herkese selamlar ben savaş dönmez bugün ...,OhMefsPMa8A,https://www.youtube.com/watch?v=OhMefsPMa8A&t=...


In [62]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [63]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
text = df_word_group_time_loc_in_text_result.loc[13,"sentence"]
text

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
sorted(all_index_list, key=len, reverse=True)

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

In [ ]:
df_adjust_text_ratio

In [ ]:
df_test = df_adjust_text_ratio.iloc[4:5,:]
#df_test.reset_index(drop=True, inplace=True)
df_test

In [ ]:
# tested
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [ ]:
df_new = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=True, sent_len_num=10)
df_new

In [ ]:
df_new.drop_duplicates()

In [ ]:
#text = df_videoid_sentence.loc[0,"sentence"]
#text

In [ ]:
text = "ablan burada nerede i çeriye kadar gelemedi ama benimle buraya kadar geldi sen şimdi git baba sonra beni almaya gel ama hemen gel tamam mı tamam defol defol defol her şeyi mahvettin defol ne işin var burada hayır hayır hayır bırakın bırak beni bırak beni bırak"

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
len(words)

In [ ]:
words[1] in word_list

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
index_list

In [ ]:
index_list[0] 

In [ ]:
len(index_list)

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
all_index_list[0]

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text